In [ ]:
import numpy as np
import json
from typing import Dict, List, Any
from pathlib import Path
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from modelscope import AutoModelForCausalLM, AutoTokenizer

In [ ]:
torch.empty_cache()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.eval()

In [ ]:
class DataLoader:
    def __init__(self, data_dir: str):
        self.data_dir = Path(data_dir)
    
    def load_tasks(self, split: str = "training") -> List[Dict[str, Any]]:
        tasks = []
        file_pattern = self.data_dir / split / "*.json"
        
        for file_path in glob.glob(str(file_pattern)):
            with open(file_path, "r") as f:
                task_data = json.load(f)
                task = {
                    "task_id": Path(file_path).stem,
                    "train": task_data["train"],
                    "test": task_data["test"]
                }
                tasks.append(task)
        if not tasks:
            raise FileNotFoundError(f"No JSON files found in {file_pattern}")
        return tasks
    
    def augment_data(self, task: Dict[str, Any]) -> Dict[str, Any]:
        # Placeholder for data augmentation logic
        # This could include transformations, noise addition, etc.
        augmented_task = task.copy()
        return augmented_task

In [ ]:
class Prompter:
    def __init__(self, template: str = None):
        self.template = template or """
Task Description:
Given input-output grid pairs, identify the transformation pattern and apply it to the test input.

Training Examples:
{train_examples}

Test Input:
{test_input}

Please provide the output grid for the test input based on the pattern observed in the training examples.
"""

    def format_train_example(self, example: Dict[str, List[List[int]]]) -> str:
        input_grid = np.array(example["input"])
        output_grid = np.array(example["output"])
        return f"Input:\n{input_grid.tolist()}\nOutput:\n{output_grid.tolist()}\n"

    def create_prompt(self, task: Dict[str, Any]) -> str:
        train_examples = ""
        for example in task["train"]:
            train_examples += self.format_train_example(example) + "\n"
        
        test_input = np.array(task["test"][0]["input"]).tolist()
        
        return self.template.format(
            train_examples=train_examples,
            test_input=test_input
        )

In [ ]:
class Solver:
    def __init__(self, model: nn.Module):
        self.model = model
        self.model.to(device)
    
    def solve(self, prompt: str) -> List[List[int]]:
        # Model inference logic would go here
        # For now, we return a dummy output
        return [[0, 0], [0, 0]]

In [ ]:
class Evaluator:
    def __init__(self):
        self.total_tasks = 0
        self.correct_tasks = 0
    
    def evaluate(self, predicted: List[List[int]], ground_truth: List[List[int]]) -> bool:
        predicted = np.array(predicted)
        ground_truth = np.array(ground_truth)
        return np.array_equal(predicted, ground_truth)
    
    def update_metrics(self, task: Dict[str, Any], prediction: List[List[int]]):
        self.total_tasks += 1
        ground_truth = task["test"][0].get("output")
        if ground_truth and self.evaluate(prediction, ground_truth):
            self.correct_tasks += 1
            
    def reset_metrics(self):
        self.total_tasks = 0
        self.correct_tasks = 0
    
    def get_accuracy(self) -> float:
        return self.correct_tasks / self.total_tasks if self.total_tasks > 0 else 0.0

In [ ]:
class Trainer:
    def __init__(self, data_loader: DataLoader, prompter: Prompter, solver: Solver, evaluator: Evaluator):
        self.data_loader = data_loader
        self.prompter = prompter
        self.solver = solver
        self.evaluator = evaluator
        self.model = solver.model
    
    def train(self, split: str = "training", epochs: int = 1):
        # TODO: Implement training logic
        tasks = self.data_loader.load_tasks(split)
        for epoch in range(epochs):
            print(f"Epoch {epoch + 1}/{epochs}")
            for task in tasks:
                augmented_task = self.data_loader.augment_data(task)
                prompt = self.prompter.create_prompt(augmented_task)
                prediction = self.solver.solve(prompt)
                self.evaluator.update_metrics(task, prediction)
                
            accuracy = self.evaluator.get_accuracy()
            print(f"Accuracy after epoch {epoch + 1}: {accuracy:.2f}")
            self.evaluator.reset_metrics()

In [ ]:
class ARC:
    def __init__(self, data_loader: DataLoader, prompter: Prompter, solver: Solver, evaluator: Evaluator):
        self.data_loader = data_loader
        self.prompter = prompter
        self.solver = solver
        self.evaluator = evaluator
        self.trainer = Trainer(data_loader, prompter, solver, evaluator)
    
    def arc_score(self):
        self.trainer.train()
        

In [ ]:
data_loader = DataLoader(data_dir="data")
prompter = Prompter()
solver = Solver(model)
evaluator = Evaluator()
arc = ARC(data_loader, prompter, solver, evaluator)
arc.arc_score()